# Creator Metrics

This script processes a CSV from [this Dune Query](https://dune.com/queries/4087193) and derives metrics the following NFT creator metrics:

- num drops
- num unique minters
- num txns
- USD value of txns

In [1]:
from dotenv import load_dotenv
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import json
import os
import pandas as pd

2024-09-28 19:13:42,779 INFO numexpr.utils Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-09-28 19:13:42,786 INFO numexpr.utils NumExpr defaulting to 8 threads.


In [2]:
load_dotenv()

DUNE_API_KEY = os.getenv('DUNE_API_KEY')
dune = DuneClient(DUNE_API_KEY)

## Part 1. Parse the creator data

In [3]:
apps = pd.read_csv('data/apps/creator_apps.csv', index_col=0)
apps = apps.drop(columns=['chain']).drop_duplicates()
uuid_to_address_mapper = apps.set_index('uuid')['address'].to_dict()
uuid_list = list(apps['uuid'].unique())
address_list = list(apps['address'].unique())
apps.tail(1)

,uuid,recipient,project_type,category,address
1362,568706f3-8127-448f-b0b1-56c28e4f3ed1,0xf56E55e35d2CCa5A34F5Ba568454974424aEA0F4,Creator,Art NFTs,0xf56e55e35d2cca5a34f5ba568454974424aea0f4


In [4]:
all_apps = pd.read_csv('data/apps/applications_reviewed.csv')
all_apps = all_apps[all_apps['uuid'].isin(uuid_list)]
uuid_to_profile_mapper = all_apps.set_index('uuid')['profile_name'].to_dict()
uuid_to_name_mapper = all_apps.set_index('uuid')['name'].to_dict()
uuid_to_id_mapper = all_apps.set_index('uuid')['id'].to_dict()
all_apps.tail(1)

,uuid,charmverseId,agoraProjectRefUID,id,recipient,time,name,status,profile_name,profile_url,...,chain_id,chain,flag_multiple_projects_same_profile,flag_creator_no_address,flag_app_missing_contract,flag_channel_no_channel,flag_charmverse_in_name,flag_creator_address_conflict,count_flags,has_flag
1475,568706f3-8127-448f-b0b1-56c28e4f3ed1,568706f3-8127-448f-b0b1-56c28e4f3ed1,0x2b24be9c82b7cbd266af7a2a27fbd3213c98bd93c01d...,0xb07fdf1ede14ea0abf845c015d4ad73f336e50acb273...,0xf56E55e35d2CCa5A34F5Ba568454974424aEA0F4,1725398873,面白い事実,pending,music-guy.eth,https://warpcast.com/music-guy.eth,...,NaN,All Superchain,False,False,False,False,False,False,0,0


In [5]:
app_dict = {}
for _,app in apps.iterrows():
    recipient = app['recipient'].lower()
    uuid = app['uuid']
    minting_address = app['address'].lower()
    if recipient not in app_dict:
        app_dict.update({
            recipient: {
                'mintingAddressList': [minting_address],
                'uuidList': [uuid]
            }
        })
    else:
        app_dict[recipient]['mintingAddressList'].append(minting_address)
        app_dict[recipient]['uuidList'].append(uuid)

app_json = []        
for recipient, record in app_dict.items():
    app_json.append({
        'recipientAddress': recipient,
        'mintingAddressList': list(set(record['mintingAddressList'])),
        'uuidList': list(set(record['uuidList'])),
    })
        
with open("data/apps/creator_address_to_uuid.json", "w") as f:
    json.dump(app_json, f, indent=2)

## Part 2. Grab NFT mint data from Dune

In [6]:
creator_addresses_str = ',\n\t\t'.join(address_list)
query_sql = f"""
    select
        blockchain,
        project,
        buyer,
        tx_to,
        tx_from,
        nft_contract_address,
        project_contract_address,
        coalesce(amount_usd, 0) as amount_usd,
        number_of_items
    from nft.mints
        where
            block_date between date('2024-06-01') and date('2024-09-01')
            and blockchain in ('base', 'optimism', 'zora')
            and tx_from in  (
                {creator_addresses_str}
            )
"""

In [7]:
# query_id = dune.create_query(name='sunny_nft_mints', query_sql=query_sql, is_private=False)
# query = QueryBase(name='sunny_nft_mints', query_id=query_id.base.query_id)
# results_df = dune.run_query_dataframe(query)
# results_df.to_parquet('data/raw_metric_data/dune_raw_nft_mints.parquet')
nft_creators = pd.read_parquet('data/raw_metric_data/dune_raw_nft_mints.parquet')
nft_creators.rename(columns={'tx_from': 'creator_address'}, inplace=True)
nft_creators['blockchain'] = nft_creators['blockchain'].apply(lambda x: x.title())
nft_creators.tail()

,blockchain,project,buyer,tx_to,creator_address,nft_contract_address,project_contract_address,amount_usd,number_of_items
56075,Base,basecolors,0x00409fc839a2ec2e6d12305423d37cd011279c09,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0x00409fc839a2ec2e6d12305423d37cd011279c09,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,3.427170,1
56076,Base,Unknown,0x1a1c37c145a1eab58c43f003ebb55c18083b5987,0xad14d16d5e980900d5d87153b1de44bd8b02892d,0x1a1c37c145a1eab58c43f003ebb55c18083b5987,0xad14d16d5e980900d5d87153b1de44bd8b02892d,0xad14d16d5e980900d5d87153b1de44bd8b02892d,0.328370,1
56077,Base,basecolors,0xd91c4283ebbc00af162b73418ec4ab0b3c159900,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0xd91c4283ebbc00af162b73418ec4ab0b3c159900,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0x7bc1c072742d8391817eb4eb2317f98dc72c61db,0.000000,1
56078,Base,Unknown,0x5d85e0403f815ce6b88fa7653ee84e46e5f59b95,0xfee588791cda1d01ccfc80b51efa00c0be5b129e,0x5d85e0403f815ce6b88fa7653ee84e46e5f59b95,0x1d2550d198197df1a10af515cf2ea0d790889b93,0xfee588791cda1d01ccfc80b51efa00c0be5b129e,2.771984,1
56079,Base,zora,0x4412e1fd47b780c59976694a778999698e1fe3a8,0x1bbea2cc3b2c41774687fd00684ad12f9a666557,0x589ffbbda0eacd5a9c2ba208b379c886b2630503,0x1bbea2cc3b2c41774687fd00684ad12f9a666557,0x1bbea2cc3b2c41774687fd00684ad12f9a666557,0.000000,1


In [8]:
farcaster_df = pd.read_parquet('data/raw_metric_data/farcaster.parquet')
fids = farcaster_df.set_index('address')['fid'].to_dict()
nft_creators['buyer_fid'] = nft_creators['buyer'].map(fids)

nft_creators['all_fids'] = (
    nft_creators[['buyer', 'tx_to']]
    .map(fids.get)
    .apply(
        lambda row: [fid for fid in row if fid is not None]
        ,axis=1)
)

In [9]:
nft_creators.groupby('creator_address')['all_fids'].agg(lambda x: len(set(sum(x, []))))

creator_address
0x00409fc839a2ec2e6d12305423d37cd011279c09      6
0x05a7eb929209f5c1f2f08dd087b64bb1beba99fe      1
0x05e0ccdee8e642388da4478facc6ac7c496714f6      1
0x07bd4932d08e5956fab224965a58bf1a441134a1    102
0x083c66abcf9c82dedb6d75e1242e6bda7d28aea9      1
                                             ... 
0xfb2fee25a4bc47ef183fdb5ee97d250f1fdcb69e      1
0xfc739e6188f0b21246c13a1f3da6819709f467e9      1
0xfc9307d74c87a48b504e74f92b034b1484561d9f      6
0xfe952def1c0e295048374ccd0b8ebe7001f1123e      1
0xfffcd4a86c341577316f082475826c957d6ba129      1
Name: all_fids, Length: 288, dtype: int64

# Part 3. Derive the metrics

In [10]:
metrics_by_address = pd.concat([
    nft_creators.groupby('creator_address')['nft_contract_address'].nunique().rename('num_drops'),
    nft_creators.groupby('creator_address')['buyer'].nunique().rename('num_unique_minters'),
    nft_creators.groupby('creator_address')['buyer'].count().rename('num_transactions'),
    nft_creators.groupby('creator_address')['amount_usd'].sum().rename('usd_value_of_transactions'),
    nft_creators.groupby('creator_address')['buyer_fid'].nunique().rename('num_farcaster_minters'),
    nft_creators.groupby('creator_address')['all_fids'].agg(lambda x: len(set().union(*x))).rename('num_farcaster_transactions')
], axis=1)

In [11]:
metrics = []
errors = []
for uuid, app in apps.set_index('uuid').iterrows():
    
    name = uuid_to_name_mapper.get(uuid)
    address = app['address']
    recipient = app['recipient']
    
    if address not in metrics_by_address.index:
        errors.append({
            'uuid': uuid,
            **app,
            'status': 'no_nft_activity_90D'
        })
        continue
    
    creator_metrics = metrics_by_address.loc[address].to_dict()
    
    metrics.append({
        'uuid': uuid,
        **app,
        'status': 'metrics_available',
        **creator_metrics,
    })

In [12]:
df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv('data/clean_metric_data/metrics_creators.csv')

## Part 4. Derive metrics by recipient address

In [13]:
recipient_mapping = (
    all_apps
    .groupby(['recipient', 'profile_name'])
    ['address']
    .unique()
    .apply(list)
    .to_dict()
)

In [14]:
recipient_metrics = []
for (recipient,profile_name),address_list in recipient_mapping.items():

    dff = nft_creators[nft_creators.creator_address.isin(address_list)]
    if len(dff) < 1:
        continue

    uuid_list =  all_apps[(all_apps['recipient'] == recipient) & (all_apps['profile_name'] == profile_name)]['uuid'].unique()
    app_id_list = list(set([uuid_to_id_mapper[x] for x in uuid_list]))
    name_list = list(set([uuid_to_name_mapper[x] for x in uuid_list]))

    creator_metrics = {
        'recipient': recipient,
        'artist_profile': profile_name,
    
        'num_drops': dff['nft_contract_address'].nunique(),
        'num_unique_minters': dff['buyer'].nunique(),
        'num_transactions': dff['buyer'].count(),
        'usd_value_of_transactions': dff['amount_usd'].sum(),
        'num_farcaster_minters': dff['buyer_fid'].nunique(),
        'num_farcaster_transactions': dff['all_fids'].agg(lambda x: len(set().union(*x))),
        
        'uuid_list': uuid_list,
        'application_id_list': app_id_list,
        'project_names': name_list
    }
    recipient_metrics.append(creator_metrics)

In [15]:
df_recipient_metrics = pd.DataFrame(recipient_metrics).set_index(['recipient', 'artist_profile'])
df_recipient_metrics.to_csv('data/clean_metric_data/metrics_creators_by_recipient.csv')
df_recipient_metrics.sort_values(by='num_transactions')

,,num_drops,num_unique_minters,num_transactions,usd_value_of_transactions,num_farcaster_minters,num_farcaster_transactions,uuid_list,application_id_list,project_names
recipient,artist_profile,,,,,,,,,
0xaC362A1AC836a72134972426766Fb928022Aab10,abdul11l,1,1,1,0.193244,1,1,[6752a14f-0218-4af3-84cc-4c1ecfc56b99],[0x476aa3ed77b012ed2d0dc322ab954622ee7a04eb98f...,[Mint Garden]
0x2aEFa2502B1fcB16B8091c31Ce7C70e17f0172Ba,basebuilder,1,1,1,0.290983,1,1,[50ea9d76-9fa9-4e21-b698-32108f4ad411],[0xd0977c9a1ed81a1dd0f55837ad1c641c64d6594ffa0...,[Base Builders]
0x056d8e22759a9aE9359A21749c6D7d6C12d97177,dnznjuan,1,1,1,2.584701,0,1,"[cca26bbf-8bda-401a-a119-4614c6e84ca5, 5fe954e...",[0x354f1de0f2a606b7ef8288a427345f10c896b757de1...,"[Denizen Juan on Base, Early Works Collection,..."
0xE1e5dcbBc95aabE80E2f9c65C7A2cEF85daF61C4,liliop.eth,1,1,1,2.962802,1,1,[a5c86799-7937-412d-a246-d19b13accfa1],[0x3a340a6d7842691bd53183a46b593edf0a85a005265...,[Liliop.eth sunnies nails & mirror blogs]
0xbACB708162D8Ae383c267b50D578B70c357a1654,polygon1993,1,1,1,6.310290,0,1,[d87979f9-9b93-43a2-bb2e-cb8dd7835501],[0xdca4d16538c0cc40dff28be5ac33109ad784c694b58...,[ＡＺＵＲＥ]
...,...,...,...,...,...,...,...,...,...,...
0x702ba46435D1E55B18440100BC81EB055574875e,panik,47,2,2058,393.085791,1,2,"[6476b680-2ea8-4d07-9a1b-226dc70d200a, 92dbc54...",[0x4013596428b12556146012741412595dcf698c86d4e...,"[moOnchain Summer, Based Whorls]"
0xCEEd9585854F12F81A0103861b83b995A64AD915,nounishprof,11,121,2355,24.009700,112,164,"[7528bd62-8e2b-42f1-afbf-f7c922b63390, 53cf1ce...",[0xf13063e0d3196b0a7065f7e9767a5b1306461d76959...,"[Here for the Art, CastOut]"
0x589FFBbdA0EaCD5A9C2BA208b379c886B2630503,thepark,194,270,2525,2645.427699,138,1244,[21f12c2a-ff53-4a19-b2ad-e57c09394480],[0xd11a7fef9c675e8d32b1d1d3d2ede96a82b1baa84b2...,[fridays at the park season 002]
